In [1]:
import sklearn.datasets as datasets
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = datasets.fetch_california_housing()
X = df.data
y = df.target

X, y

(array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]))

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

#### Создать Dataset для загрузки данных

In [41]:
class HousingDataset(torch.utils.data.Dataset):
   
    def __init__(self, data, target):
        self._data = data
        self._target = target

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        return self._data[idx], self._target[idx]

In [65]:
train_ds = HousingDataset(X_train, y_train)

#### Обернуть его в Dataloader

In [71]:
batch_size = 10

In [72]:
train_loader = torch.utils.data.DataLoader(train_ds,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
# при num_workers > 0 выдаёт ошибку

In [57]:
#for i, batch in enumerate(train_loader):
#    print(i, batch)

#### Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

In [82]:
import torch.nn.functional as F
import torch.nn as nn

class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        #self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        #self.bn2 = nn.BatchNorm1d(hidden_dim)
        #self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 10, "relu")
        
    def forward(self, x):
        #x = self.bn1(x)
        x = self.fc1(x)
        #x = self.bn2(x)
        #x = self.dp(x)
#         x = self.fc2(x)
        return x

In [83]:
features_len = len(df['feature_names'])

In [84]:
multiplier = 1

In [85]:
net = FeedForward(batch_size * multiplier, features_len * multiplier)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [86]:
from tqdm import tqdm

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x8 and 10x8)